In [1]:
import json
import pandas as pd

In [2]:
#通过“抓取12306上火车站点英文缩写”获取站点信息
text = open(r'./ceshi_result.txt','r').readlines()

In [9]:
#数组或对象之中的字符串必须使用双引号，不能使用单引号
text_use = text[0].replace('\'','\"')
zd = json.loads(text_use)

In [45]:
#arrow格式转换
import arrow
arrow.now().format('YYYY-MM-DD HH:mm:ss')

'2020-04-15 16:46:39'

In [33]:
#根据字典的value找key
list(zd.keys())[list(zd.values()).index('ZJH')]

'镇江'

In [28]:
list(zd.values()).index('SZH')

380

In [34]:
hu_ning = [zd['上海'],zd['昆山南'],zd['苏州'],zd['无锡'],zd['常州'],zd['丹阳'],zd['镇江'],zd['南京']]
hu_ning

['SHH', 'KNH', 'SZH', 'WXH', 'CZH', 'DYH', 'ZJH', 'NJH']

In [46]:
#对站点进行两两组合
from itertools import combinations

com = combinations(hu_ning,2)
list(com)[:3]

[('SHH', 'KNH'), ('SHH', 'SZH'), ('SHH', 'WXH')]

#### 代码优化

In [55]:
import requests
import json
import pandas
import arrow
from itertools import combinations
import time

class HighSpeed(object):
    #定义实例化后的初始参数
    def __init__(self,date,from_station,to_station):
        self.date = date
        self.from_station = from_station
        self.to_station = to_station
        
    def get_pd(self):
        headers={
        "Accept": "text/html, application/xhtml+xml, application/xml; q=0.9, */*; q=0.8",
        "Connection": "Keep-Alive",
        "Cookie": "JSESSIONID=CE1E0910299807B0B5D6CE1AE4B37318; RAIL_DEVICEID=KIGorQUspNUQqHtpHqI4qcn_7n8neHBX58xwMxWqPc26JQIimw-uxozqOZEOb_cizOpzFiNc04ZF_sK3raRaFjH3Ao-IO7KIEkB24NnWaD_4ME1YfS8KrNhz637Z2NQIjcjzvKENUq6HfF0Thy8UHfZpscWtyFJf; RAIL_EXPIRATION=1587115346591; route=6f50b51faa11b987e576cdb301e545c4; BIGipServerotn=1490616586.50210.0000; _jc_save_toStation=%u5170%u5DDE%2CLZJ; _jc_save_wfdc_flag=dc; _jc_save_toDate=2020-04-14; _jc_save_fromDate=2020-04-14; _jc_save_fromStation=%u897F%u5B89%2CXAY",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18362"
        }
        url = 'https://kyfw.12306.cn/otn/leftTicket/query?leftTicketDTO.train_date='\
        +self.date+'&leftTicketDTO.from_station='+self.from_station+'&leftTicketDTO.to_station='+self.to_station+'&purpose_codes=ADULT'

        text = requests.get(url,headers=headers).text
        return text

    def to_csv(self):
        text = self.get_pd()
        js = json.loads(text)
        data = js['data']['result']
        dt_all = []
        for dt in data:
            dt_all.append(dt.split('|'))

        df_use = pd.DataFrame(dt_all)
        df_use = df_use[df_use[3].str.contains('D') | df_use[3].str.contains('G')]
        df_use = df_use[[2,3,6,7,8,9,22,26,30,31,32,33]]
        df_use = df_use.rename(columns={2:'编号',3:'车次',6:'发车站',7:'到达站',
                                    8:'发车时间',9:'到达时间',22:'其他',26:'无座',
                                    30:'二等座',31:'一等座',32:'商务特等座',33:'动卧'})
        df_use.replace('',0,inplace=True)
        tm = arrow.now().format('HH:mm')
        df_use['record_time'] = tm
        print('{} to {} finished!'.format(self.from_station,self.to_station))
        
        df_use.to_csv(r'../../file/subject/sj2_data{}.csv'.format(self.date),mode='a',index=False)
        time.sleep(2)

if __name__ == '__main__':
    date = arrow.now().format('YYYY-MM-DD')
    hu_ning = ['SHH', 'KNH', 'SZH', 'WXH', 'CZH', 'DYH', 'ZJH', 'NJH']
#     hu_ning = ['SHH', 'KNH', 'SZH']
    com = combinations(hu_ning,2)
    for ls in com:
        hspeed = HighSpeed(date,ls[0],ls[1])
        hspeed.to_csv()
    print('finished!')

SHH to KNH finished!
SHH to SZH finished!
SHH to WXH finished!
SHH to CZH finished!
SHH to DYH finished!
SHH to ZJH finished!
SHH to NJH finished!
KNH to SZH finished!
KNH to WXH finished!
KNH to CZH finished!
KNH to DYH finished!
KNH to ZJH finished!
KNH to NJH finished!
SZH to WXH finished!
SZH to CZH finished!
SZH to DYH finished!
SZH to ZJH finished!
SZH to NJH finished!
WXH to CZH finished!
WXH to DYH finished!
WXH to ZJH finished!
WXH to NJH finished!
CZH to DYH finished!
CZH to ZJH finished!
CZH to NJH finished!
DYH to ZJH finished!
DYH to NJH finished!
ZJH to NJH finished!
finished!
